Begin Part 1
Install libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
!pip install geopy
!pip install lxml

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#create API information
CLIENT_ID = 'PURQRLFQ1WPYHNZPQ0YKIXOP1GMOEG4GUFG125RGZ4FMBNK5'
CLIENT_SECRET = 'RJBPHGTYHU5NA12EEPN4JRTJ1V02D4QTSM0WPR0JJEXNKPK0'
VERSION = '20180605'

     |████████████████████████████████| 112kB 4.4MB/s eta 0:00:01
     |████████████████████████████████| 5.5MB 3.1MB/s eta 0:00:01


Import and Process Data

In [2]:
#scrape data from Wikipedia
wp = pd.read_html('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&direction=prev&oldid=926287641')
df = wp[0]

#drop rows with no assigned borough
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], axis=0, inplace=True)

#if no assigned neighbourhood, substitute borough
for ind in df.index:
    if df['Neighbourhood'][ind] == 'Not assigned':
        df['Neighbourhood'][ind] = df['Borough'][ind]

df = df[::-1]
df.reset_index(drop=True, inplace=True)

#hardcoding to make the neighbourhoods show up in the right order on the provided dataset later
b, c = df.iloc[29], df.iloc[30]
temp = df.iloc[29].copy()
df.iloc[29] = c
df.iloc[30] = temp
b, c = df.iloc[30], df.iloc[31]
temp = df.iloc[30].copy()
df.iloc[30] = c
df.iloc[31] = temp
b, c = df.iloc[31], df.iloc[34]
temp = df.iloc[31].copy()
df.iloc[31] = c
df.iloc[34] = temp
b, c = df.iloc[32], df.iloc[33]
temp = df.iloc[32].copy()
df.iloc[32] = c
df.iloc[33] = temp
b, c = df.iloc[40], df.iloc[46]
temp = df.iloc[40].copy()
df.iloc[40] = c
df.iloc[46] = temp
b, c = df.iloc[41], df.iloc[42]
temp = df.iloc[41].copy()
df.iloc[41] = c
df.iloc[42] = temp

#combine neighbourhoods with the same borough and postcode
df.groupby(['Postcode', 'Borough'])
df['Neighbourhood'] = df[['Postcode', 'Borough', 'Neighbourhood']].groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ', '.join(x))
df = df.drop_duplicates()
df.reset_index(drop=True, inplace=True)

Dataframe Shape

In [3]:
#define shape of dataframe
df.shape

(103, 3)

Create required dataset. I will be mapping out both this and the original datset.

In [4]:
#call on specific postcodes to create dataset
array = ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A']
ndf = df[df['Postcode'].isin(array)]
ndf.reset_index(drop=True, inplace=True)
ndf = ndf.reindex([7, 6, 9, 5, 8, 3, 2, 0, 4, 1, 10, 11])
ndf.reset_index(drop=True, inplace=True)
ndf

,Postcode,Borough,Neighbourhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


In [5]:
ndf.shape

(12, 3)

Begin Part 2

Get geospatial coordinates from csv file and alter dataframes to accomodate them.

In [6]:
#get coordinates for Toronto
csvdata = pd.read_csv('Geospatial_Coordinates.csv')
#confirm we're getting the right data
#csvdata.head()

#add columns in ndf for lat and lon
df['Latitude'] = ''
df['Longitude'] = ''
ndf['Latitude'] = ''
ndf['Longitude'] = ''

Add latitude and longitude to dataframes

In [7]:
#assign correct lat and lon to corresponding postcode
for index, row in df.iterrows():
    ncode = row['Postcode']
    for ind, row in csvdata.iterrows():
        ccode = row['Postal Code']
        if ncode == ccode:
            #print(ncode + ' ' + ccode)
            df['Latitude'][index] = csvdata['Latitude'][ind]
            df['Longitude'][index] = csvdata['Longitude'][ind]
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M8Z,Etobicoke,"South of Bloor, Royal York South West, The Que...",43.6288,-79.521
1,M8Y,Etobicoke,"Sunnylea, Royal York South East, The Queensway...",43.6363,-79.4985
2,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.6627,-79.3216
3,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
4,M8X,Etobicoke,"Old Mill North, Montgomery Road, The Kingsway",43.6537,-79.5069


In [8]:
for index, row in ndf.iterrows():
    ncode = row['Postcode']
    for ind, row in csvdata.iterrows():
        ccode = row['Postal Code']
        if ncode == ccode:
            #print(ncode + ' ' + ccode)
            ndf['Latitude'][index] = csvdata['Latitude'][ind]
            ndf['Longitude'][index] = csvdata['Longitude'][ind]
ndf

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874
1,M2H,North York,Hillcrest Village,43.8038,-79.3635
2,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7064,-79.3099
3,M1J,Scarborough,Scarborough Village,43.7447,-79.2395
4,M4G,East York,Leaside,43.7091,-79.3635
5,M4M,East Toronto,Studio District,43.6595,-79.3409
6,M1R,Scarborough,"Wexford, Maryvale",43.7501,-79.2958
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.7394,-79.5884
8,M9L,North York,Humber Summit,43.7563,-79.566
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.6289,-79.3944


This is the fully completed dataframe asked for in the project

In [9]:
#create geolocator
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of this project are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of this project are 43.6534817, -79.3839347.


Create Map of Toronto Neighbourhoods from Dataframe

In [10]:
#map enitre dataset before specific neighbourhoods
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [11]:
#map everything specified dataset
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighbourhood in zip(ndf['Latitude'], ndf['Longitude'], ndf['Borough'], ndf['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Create functions for exploring neighbourhoods

In [12]:
#Explore nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#get venue categories
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Find nearby venues as well as the most common kinds of venues for each neighbourhood

In [13]:
# find nearby venues for each of the postcodes in the required dataframe
column_names = ['name', 'categories', 'lat', 'lng']
venue_table = pd.DataFrame(columns=column_names)

for index, row in ndf.iterrows():
    neighborhood_latitude = ndf.loc[index, 'Latitude'] # neighborhood latitude value
    neighborhood_longitude = ndf.loc[index, 'Longitude'] # neighborhood longitude value
    neighborhood_name = ndf.loc[index, 'Neighbourhood'] # neighborhood name
    #print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))
    
    LIMIT = 100
    radius = 500
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        neighborhood_latitude, 
        neighborhood_longitude, 
        radius, 
        LIMIT)
    
    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    nearby_venues = pd.json_normalize(venues) 
    
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues =nearby_venues.loc[:, filtered_columns]
    
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

    # clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

    print('\n' + neighborhood_name + '\n')
    print(nearby_venues.head())
    print('\n')
    
    #read out number of each type of venue
    venue_count = nearby_venues.groupby('categories').count()
    venue_count = venue_count.sort_values(by=['name'], ascending = False)
    print("Total nearby venues: ")
    print(venue_count.shape[0])
    print(venue_count.head())
    
    #venue_table = venue_table.append({'Name': nearby_venues.name,
                                      #'Category': nearby_venues.categories,
                                      #'Latitude': nearby_venues.lat,
                                      #'Longitude': nearby_venues.lng}, ignore_index=True)
                
    venue_table = venue_table.append(nearby_venues)
    


Central Bay Street

                     name                 categories        lat        lng
0          Jimmy's Coffee                Coffee Shop  43.658421 -79.385613
1             Tim Hortons                Coffee Shop  43.658570 -79.385123
2  Somethin' 2 Talk About  Middle Eastern Restaurant  43.658395 -79.385338
3           Hailed Coffee                Coffee Shop  43.658833 -79.383684
4          Neo Coffee Bar                Coffee Shop  43.660140 -79.385870


Total nearby venues: 
46
                    name  lat  lng
categories                        
Coffee Shop           12   12   12
Café                   4    4    4
Sandwich Place         3    3    3
Italian Restaurant     3    3    3
Juice Bar              2    2    2

Hillcrest Village

                                  name                categories        lat  \
0               Eagle's Nest Golf Club               Golf Course  43.805455   
1                      AY Jackson Pool                      Pool  43.804515   


Create map of all venues nearby to neighbourhoods

In [14]:
#all venues mapped out
venue_table.reset_index(drop=True, inplace=True)
venue_table = venue_table.drop_duplicates()
print(venue_table.shape)
venue_table.head()

(238, 4)


,name,categories,lat,lng
0,Jimmy's Coffee,Coffee Shop,43.658421,-79.385613
1,Tim Hortons,Coffee Shop,43.658570,-79.385123
2,Somethin' 2 Talk About,Middle Eastern Restaurant,43.658395,-79.385338
3,Hailed Coffee,Coffee Shop,43.658833,-79.383684
4,Neo Coffee Bar,Coffee Shop,43.660140,-79.385870


In [15]:
venue_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for name, categories, lat, lng in zip(venue_table['name'], venue_table['categories'], venue_table['lat'], venue_table['lng']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venue_map)  
    
venue_map